In [ ]:
"""
Filter the ACS dataset to match LBS data area
Make choropleth map for home location density -- for later visual comparison to LBS data

Smaller steps:
- read in ACS dataset
- read in shapefile
- prune the files to only include columns of interest, rename columns to be readable
- join the files on census area
"""

import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# for saving the map figures
DPI = 300
output_dir = './results/maps/'

GEOGRAPHY = 'Geography'

COUNTY_SUBDIVISION_GEOID = "COUNTY SUBDIV GEOID"
TRACT_GEOID = 'TRACT GEOID'
BLOCKGROUP_GEOID = 'BLOCKGROUP GEOID'


# Population density for county subdivision

In [ ]:
acs_census_county_subdivisions_filepath = "./data/ACS/ma_acs_5_year_census_county_subdivisions_2017/ACS_17_5YR_B01003_with_ann.csv"
acs_census_county_subdivisions_df = pd.read_csv(acs_census_county_subdivisions_filepath)
print(acs_census_county_subdivisions_df.shape)
acs_census_county_subdivisions_df.head(3)

In [ ]:
# drop the Id column because it is unneeded.
acs_census_county_subdivisions_df.drop(columns=['Id'], inplace=True)
# rename important geoid column that will be used for merge/join
acs_census_county_subdivisions_df.rename(columns={'Id2': COUNTY_SUBDIVISION_GEOID}, inplace=True)
print(acs_census_county_subdivisions_df.shape)
acs_census_county_subdivisions_df.head()

In [ ]:
# Read in shapefile
census_county_subdivisions_shapefile_filepath = "./shapefiles/ma/middlesex-suffolk-norfolk-countysubdivisions.shp"
census_county_subdivisions_shapefile = geopandas.read_file(census_county_subdivisions_shapefile_filepath)
print(census_county_subdivisions_shapefile.shape)
census_county_subdivisions_shapefile.head(5)

In [ ]:
# Update the GEOID column to match the ACS data 
census_county_subdivisions_shapefile.rename(columns={'GEOID': COUNTY_SUBDIVISION_GEOID}, inplace=True)
census_county_subdivisions_shapefile[COUNTY_SUBDIVISION_GEOID] = census_county_subdivisions_shapefile.astype({COUNTY_SUBDIVISION_GEOID: 'int64'})[COUNTY_SUBDIVISION_GEOID]
# Merge the population data with the shapefile
census_county_subdivisions_shapefile_data_merged = census_county_subdivisions_shapefile.set_index(COUNTY_SUBDIVISION_GEOID).join(acs_census_county_subdivisions_df.set_index(COUNTY_SUBDIVISION_GEOID))
census_county_subdivisions_shapefile_data_merged.sort_values(by=['Estimate; Total'], ascending=False, inplace=True)
# Add a variable for population density
AREA = "ALAND"
POPULATION_ESTIMATE = "Estimate; Total"
POPULATION_ESTIMATE_DENSITY = "POP_ESTIMATE_DENSITY"


census_county_subdivisions_shapefile_data_merged[POPULATION_ESTIMATE_DENSITY] = census_county_subdivisions_shapefile_data_merged[POPULATION_ESTIMATE]/census_county_subdivisions_shapefile_data_merged[AREA]
census_county_subdivisions_shapefile_data_merged.dropna(inplace=True)

# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(15, 10))
# create map
census_county_subdivisions_shapefile_data_merged.plot(column=POPULATION_ESTIMATE_DENSITY, legend=True, ax=ax) #, cmap='Blues', linewidth=0.8, edgecolor='0.8')
# remove the axis
ax.axis('off')
# add a title
title = 'Population density by county subdivision'
ax.set_title(title, fontdict={'fontsize': '25', 'fontweight' : '3'})
# create an annotation for the data source
annotation = 'Source: ACS 5-year estimates, 2017'
ax.annotate(annotation, xy=(0.1, .08),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')
census_county_subdivisions_shapefile_data_merged.head()

In [ ]:
# Make and save a pruned version of this file that has only
# boston, brookline, cambridge, somerville
keep_names = ['boston', 'brookline', 'cambridge', 'somerville']
bos_brook_camb_som_county_subdivisions_shapefile_data = census_county_subdivisions_shapefile_data_merged[census_county_subdivisions_shapefile_data_merged['NAME'].str.lower().isin(keep_names)]
# Save this pruned map
output_filepath = "./shapefiles/ma/boston-brookline-cambridge-somerville_countysubdivisions.shp"
bos_brook_camb_som_county_subdivisions_shapefile_data.to_file(output_filepath)

# map it.  Create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(15, 10))
# create map
bos_brook_camb_som_county_subdivisions_shapefile_data.plot(column=POPULATION_ESTIMATE_DENSITY, legend=True, ax=ax) #, cmap='Blues', linewidth=0.8, edgecolor='0.8')
# remove the axis
ax.axis('off')
# add a title
title = 'Population density by county subdivision'
subtitle = '(Boston, Brookline, Cambridge, Somerville)'
fig.suptitle(title + '\n' + subtitle, fontsize=18)
# ax.set_title(title, fontdict={'fontsize': '18', 'fontweight' : '3'})
# create an annotation for the data source
annotation = 'Source: ACS 5-year estimates, 2017'
ax.annotate(annotation, xy=(0.1, .08),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')
bos_brook_camb_som_county_subdivisions_shapefile_data.head()

# Census Tracts

In [ ]:
acs_census_tracts_data_filepath = "./data/ACS/ma_acs_5_year_census_tracts_2017/ACS_17_5YR_B01003_with_ann.csv"
acs_census_tracts_df = pd.read_csv(acs_census_tracts_data_filepath)
acs_census_tracts_df.head(3)

In [ ]:
# drop the Id column because it is unneeded.
acs_census_tracts_df.drop(columns=['Id'], inplace=True)
# rename important geoid column that will be used for merge/join
acs_census_tracts_df.rename(columns={'Id2': TRACT_GEOID}, inplace=True)
acs_census_tracts_df.head(3)

In [ ]:
# Read in shapefile
# For better visualization for now, limited higher density areas close to home
census_tracts_shapefile_filepath = "./shapefiles/ma/boston-brookline-cambridge-somerville_tract.shp"
census_tracts_shapefile = geopandas.read_file(census_tracts_shapefile_filepath)
census_tracts_shapefile.plot()
census_tracts_shapefile.head(3)

In [ ]:
# Set coordinate system
census_tracts_shapefile = census_tracts_shapefile.to_crs(epsg=4326)
# Update the census tract GEOID column to match the ACS data 
census_tracts_shapefile.rename(columns={'GEOID10': TRACT_GEOID}, inplace=True)
census_tracts_shapefile[TRACT_GEOID] = census_tracts_shapefile.astype({TRACT_GEOID: 'int64'})[TRACT_GEOID]
census_tracts_shapefile.plot()
census_tracts_shapefile.head(3)

In [ ]:
merged_tracts_data_shapefile = census_tracts_shapefile.set_index(TRACT_GEOID).join(acs_census_tracts_df.set_index(TRACT_GEOID))
merged_tracts_data_shapefile.plot()
merged_tracts_data_shapefile.sort_values(by=['Estimate; Total'], ascending=False, inplace=True)
merged_tracts_data_shapefile.head(3)

In [ ]:
# Add a variable for population density
AREA_ACRES = "AREA_ACRES"
AREA_SQFT = "AREA_SQFT"
POPULATION_ESTIMATE = "Estimate; Total"
POPULATION_ESTIMATE_DENSITY_PER_ACRE = "POP_ESTIMATE_DENSITY_PER_ACRE"


merged_tracts_data_shapefile[POPULATION_ESTIMATE_DENSITY_PER_ACRE] = merged_tracts_data_shapefile[POPULATION_ESTIMATE]/merged_tracts_data_shapefile[AREA_ACRES]

# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(15, 10))
# create map
merged_tracts_data_shapefile.plot(column=POPULATION_ESTIMATE_DENSITY_PER_ACRE, legend=True, ax=ax) #, cmap='Blues', linewidth=0.8, edgecolor='0.8')
# remove the axis
ax.axis('off')
# add a title
title = 'Population density by census tract (people per acre)'
ax.set_title(title, fontdict={'fontsize': '25', 'fontweight' : '3'})
# create an annotation for the data source
annotation = 'Source: ACS 5-year estimates, 2017'
ax.annotate(annotation, xy=(0.1, .08),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')

merged_tracts_data_shapefile.head(3)

In [ ]:
# save the map
filename = 'acs-population-density-census-tract-bos-brook-cam-som.png'
filepath = output_dir + filename
print('saving map to %s' % filepath)
fig.savefig(filepath,  dpi=DPI)

# Census Block Group

In [ ]:
acs_blockgroups_data_filepath = "./data/ACS/ma_acs_5_year_census_block_group_2017/ACS_17_5YR_B02001_with_ann.csv"
acs_blockgroups_df = pd.read_csv(acs_blockgroups_data_filepath)
print(acs_blockgroups_df.shape)
acs_blockgroups_df.head(3)

In [ ]:
# drop the Id column because it is unneeded.
acs_blockgroups_df.drop(columns=['Id'], inplace=True)
# rename important geoid column that will be used for merge/join
acs_blockgroups_df.rename(columns={'Id2': BLOCKGROUP_GEOID}, inplace=True)
acs_blockgroups_df.head(3)

In [ ]:
# For now only keep race info for how many white and black people there are.
POPULATION_ESTIMATE_TOTAL = 'Estimate; Total:'
POPULATION_MARGIN_OF_ERROR_TOTAL = 'Margin of Error; Total:'
POPULATION_ESTIMATE_WHITE = 'Estimate; Total: - White alone'
POPULATION_MARGIN_OF_ERROR_WHITE = 'Margin of Error; Total: - White alone'
POPULATION_ESTIMATE_BLACK = 'Estimate; Total: - Black or African American alone'
POPULATION_MARGIN_OF_ERROR_BLACK = 'Margin of Error; Total: - Black or African American alone'
POPULATION_ESTIMATE_OTHER = 'Estimate; Total: All the others! Including mixrace households'

# For now only keep race info for how many white and black people there are.
acs_blockgroups_df = acs_blockgroups_df[[BLOCKGROUP_GEOID, GEOGRAPHY, 
                                         POPULATION_ESTIMATE_TOTAL, POPULATION_MARGIN_OF_ERROR_TOTAL, 
                                         POPULATION_ESTIMATE_WHITE, POPULATION_MARGIN_OF_ERROR_WHITE,
                                         POPULATION_ESTIMATE_BLACK, POPULATION_MARGIN_OF_ERROR_BLACK]]
# make column for everyone else...
acs_blockgroups_df[POPULATION_ESTIMATE_OTHER] = acs_blockgroups_df[POPULATION_ESTIMATE_TOTAL] - acs_blockgroups_df[POPULATION_ESTIMATE_WHITE] - acs_blockgroups_df[POPULATION_ESTIMATE_BLACK]
acs_blockgroups_df.head(3)

In [ ]:
# Read in shapefile
# For better visualization for now, limited higher density areas close to home
blockgroups_shapefile_filepath = "./shapefiles/ma/boston-brookline-cambridge-somerville_blockgroup.shp"
blockgroups_shapefile = geopandas.read_file(blockgroups_shapefile_filepath)
# Set coordinate system
blockgroups_shapefile = blockgroups_shapefile.to_crs(epsg=4326)
# Update the census tract GEOID column to match the ACS data 
blockgroups_shapefile.rename(columns={'GEOID10': BLOCKGROUP_GEOID}, inplace=True)
blockgroups_shapefile[BLOCKGROUP_GEOID] = blockgroups_shapefile.astype({BLOCKGROUP_GEOID: 'int64'})[BLOCKGROUP_GEOID]
blockgroups_shapefile.plot()
print(blockgroups_shapefile.shape)
blockgroups_shapefile.head(3)

In [ ]:
merged_blockgroups_data_shapefile = blockgroups_shapefile.set_index(BLOCKGROUP_GEOID).join(acs_blockgroups_df.set_index(BLOCKGROUP_GEOID))
merged_blockgroups_data_shapefile.plot()
fig, ax = plt.subplots(1, figsize=(15, 10))
merged_blockgroups_data_shapefile.sort_values(by=['Estimate; Total:'], ascending=False, inplace=True)
merged_blockgroups_data_shapefile.head(3)

In [ ]:
# Map population density
AREA_ACRES = "AREA_ACRES"
AREA_SQFT = "AREA_SQFT"
POPULATION_ESTIMATE = "Estimate; Total:"
# Add a variable for population density
POPULATION_ESTIMATE_DENSITY_PER_ACRE = "POP_ESTIMATE_DENSITY_PER_ACRE"


merged_blockgroups_data_shapefile[POPULATION_ESTIMATE_DENSITY_PER_ACRE] = merged_blockgroups_data_shapefile[POPULATION_ESTIMATE]/merged_blockgroups_data_shapefile[AREA_ACRES]

# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(15, 10))
# create map
merged_blockgroups_data_shapefile.plot(column=POPULATION_ESTIMATE_DENSITY_PER_ACRE, legend=True, ax=ax) #, cmap='Blues', linewidth=0.8, edgecolor='0.8')
# remove the axis
ax.axis('off')
# add a title
title = 'Population density by census block group (people per acre)'
ax.set_title(title, fontdict={'fontsize': '25', 'fontweight' : '3'})
# create an annotation for the data source
annotation = 'Source: ACS 5-year estimates, 2017'
ax.annotate(annotation, xy=(0.1, .08),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')

merged_blockgroups_data_shapefile.head(3)

In [ ]:
# save the map
filename = 'acs-population-density-census-blockgroup-bos-brook-cam-som.png'
filepath = output_dir + filename
print('saving map to %s' % filepath)
fig.savefig(filepath,  dpi=DPI)

## Compare the ACS to the LBS data

- load in the inferred homes (census block group) data (for given time period)
- create aggregate df mapping LBS inferred census area --> user count
- join this aggregated LBS data with the merged census area data shapefile
- (this should limit the data to estimates for same areas for LBS and ACS data)

- scale the LBS data:

Calculate what proportion of the area's population the LBS dataset represents. 
- LBS sample = all LBS data users with inferred homes in area
- ACS population = total population in area that was estimated by ACS/census
- proportion = (LBS sample ) / ACS population
- Scaling parameter = 1/proportion

Scale the aggregated LBS home location data by the scaling parameter to match the total population

#### Visualize:

Population density
- map the population density based on the LBS data
- compare to map of population based on ACS population estimates

Margins of error
- TODO....


In [ ]:
# - load in the inferred homes (blockgroup) data (for given time period)

# variables for dataset

DEVICE_ID = "device ID"

INFERRED_HOME_BLOCKGROUP_GEOID = "inferred home census blockgroup geoid"
USER_COUNT = "device ID count"

filepath = '../data/mount/201805/filtered/filtered_14460_201805_inferred_homes_blockgroup_bos_brook_cam_som.csv'
lbs_inferred_homes_blockgroup_df = geopandas.read_file(filepath)
print(lbs_inferred_homes_blockgroup_df.shape)
# lbs_inferred_homes_blockgroup_df.head(2)

In [ ]:
# - create aggregate df mapping LBS inferred blockgroup --> user count
# - i.e. map inferred home census area geoid to number of users
# - make DF with columns: INFERRED_HOME_BLOCKGROUP_GEOID, USER_COUNT


inferred_home_geoids = lbs_inferred_homes_blockgroup_df[INFERRED_HOME_BLOCKGROUP_GEOID].unique()
inferred_home_geoids_user_count = []
for geoid in inferred_home_geoids:
    geoid_df = lbs_inferred_homes_blockgroup_df[lbs_inferred_homes_blockgroup_df[INFERRED_HOME_BLOCKGROUP_GEOID] == geoid]
    user_count = geoid_df[DEVICE_ID].nunique()
    inferred_home_geoids_user_count.append(user_count)

lbs_inferred_homes_blockgroup_count_df = pd.DataFrame.from_dict({
    INFERRED_HOME_BLOCKGROUP_GEOID: inferred_home_geoids,
    USER_COUNT: inferred_home_geoids_user_count,
})
print(lbs_inferred_homes_blockgroup_count_df.shape)
lbs_inferred_homes_blockgroup_count_df.head()

In [ ]:
# join this aggregated LBS data with the merged census data shapefile
# (this should limit the data to estimates for same areas for LBS and ACS data)

# first must make join keys the same
lbs_inferred_homes_blockgroup_count_df[INFERRED_HOME_BLOCKGROUP_GEOID] = lbs_inferred_homes_blockgroup_count_df.astype({INFERRED_HOME_BLOCKGROUP_GEOID: 'int64'})[INFERRED_HOME_BLOCKGROUP_GEOID]

merged_acs_lbs_blockgroup_data_shapefile = merged_blockgroups_data_shapefile.join(lbs_inferred_homes_blockgroup_count_df.set_index(INFERRED_HOME_BLOCKGROUP_GEOID), how='inner')
# merged_acs_lbs_tracts_data_shapefile = lbs_inferred_homes_tract_count_df.join(merged_tracts_data_shapefile, on=INFERRED_HOME_TRACT_GEOID)
print(merged_acs_lbs_blockgroup_data_shapefile.shape)
merged_acs_lbs_blockgroup_data_shapefile.head()

In [ ]:
# scale the LBS data:

# Calculate what proportion of the area's population the LBS dataset represents.

# LBS sample = all LBS data users with inferred homes in area
# ACS population = total population in area that was estimated by ACS/census
# proportion = (LBS sample ) / ACS population
# Scaling parameter = 1/proportion

LBS_POPULATION_ESTIMATE = 'LBS population estimate'

lbs_sample_size = merged_acs_lbs_blockgroup_data_shapefile[USER_COUNT].sum()
acs_population_size = merged_acs_lbs_blockgroup_data_shapefile[POPULATION_ESTIMATE].sum()
proportion = lbs_sample_size/acs_population_size
scaling_parameter = 1/proportion
print('scaling LBS user counts by %s to get %s' % (scaling_parameter, LBS_POPULATION_ESTIMATE))
merged_acs_lbs_blockgroup_data_shapefile[LBS_POPULATION_ESTIMATE] = merged_acs_lbs_blockgroup_data_shapefile[USER_COUNT].multiply(scaling_parameter)
merged_acs_lbs_blockgroup_data_shapefile.head()

## Visualize

In [ ]:
# Add LBS estimated population density

LBS_POPULATION_ESTIMATE_DENSITY_PER_ACRE = 'LBS_POPULATION_ESTIMATE_DENSITY_PER_ACRE'

merged_acs_lbs_blockgroup_data_shapefile[LBS_POPULATION_ESTIMATE_DENSITY_PER_ACRE] = merged_acs_lbs_blockgroup_data_shapefile[LBS_POPULATION_ESTIMATE]/merged_acs_lbs_blockgroup_data_shapefile[AREA_ACRES]
# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(15, 10))
# create map
merged_acs_lbs_blockgroup_data_shapefile.plot(column=LBS_POPULATION_ESTIMATE_DENSITY_PER_ACRE, legend=True, ax=ax)
# remove the axis
ax.axis('off')
# add a title
title = 'Estimated population density by census tract'
ax.set_title(title, fontdict={'fontsize': '25', 'fontweight' : '3'})
# create an annotation for the data source
annotation = 'Source: LBS data for May 2018'
ax.annotate(annotation, xy=(0.1, .08),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')

merged_acs_lbs_blockgroup_data_shapefile.head(3)

In [ ]:
# Save this ugly map
filename = 'lbs-population-density-census-blockgroup-bos-brook-cam-som.png'
filepath = output_dir + filename
print('saving map to %s' % filepath)
fig.savefig(filepath,  dpi=DPI)

In [ ]:
# Add a column for the difference between ACS vs LBS estimates, and map it.
ESTIMATE_DIFFERENCE = 'ACS vs LBS estimate difference (blockgroup)'

merged_acs_lbs_blockgroup_data_shapefile[ESTIMATE_DIFFERENCE] = (merged_acs_lbs_blockgroup_data_shapefile[LBS_POPULATION_ESTIMATE] - merged_acs_lbs_blockgroup_data_shapefile[POPULATION_ESTIMATE])
# map the difference
# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(15, 10))
# remove the axis
ax.axis('off')
# add a title
title = 'ACS vs LBS estimate differences'
ax.set_title(title, fontdict={'fontsize': '25', 'fontweight' : '3'})
# create an annotation for the data source
annotation = 'Source: ACS 5-year 2017; LBS data for May 2018'
ax.annotate(annotation, xy=(0.1, .08),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')
merged_acs_lbs_blockgroup_data_shapefile.plot(column=ESTIMATE_DIFFERENCE, legend=True, ax=ax)
merged_acs_lbs_blockgroup_data_shapefile.head(10)

In [ ]:
# Save this ugly map
filename = 'acs-lbs-population-diff-census-blockgroup-bos-brook-cam-som.png'
filepath = output_dir + filename
print('saving map to %s' % filepath)
fig.savefig(filepath,  dpi=DPI)

What is the correlation between the ACS population estimate and the LBS user count per census area?

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Create linear regression object.
lr_home_areas = LinearRegression()

df_user_count_population_est = merged_acs_lbs_blockgroup_data_shapefile[[USER_COUNT, POPULATION_ESTIMATE]]

# Fit linear regression.
lr_home_areas.fit(df_user_count_population_est[[USER_COUNT]], df_user_count_population_est[POPULATION_ESTIMATE])

# Get the slope and intercept of the line best fit.
print(lr_home_areas.intercept_)
print(lr_home_areas.coef_)

df_user_count_population_est.head()

In [ ]:
df_user_count_population_est.corr()